In [1]:
import tensorflow as tf
tf.__version__

'2.5.0'

In [34]:
import pandas as pd
training_samples_file_path = "../../data/processed_data/train.csv/part-00000-01c21c89-bb0b-47ee-a791-64d019d8cb60-c000.csv"
test_samples_file_path = "../../data/processed_data/test.csv/part-00000-2f04f2e9-c2c8-4918-b79d-88fdee316947-c000.csv"
df = pd.read_csv(training_samples_file_path)
df_test = pd.read_csv(test_samples_file_path)

# df = pd.read_csv("../../data/trainingSamples.csv")
# df_test = pd.read_csv("../../data/testSamples.csv")


df.columns, len(df.columns), df_test.columns, len(df_test.columns)

(Index(['movieId', 'userId', 'rating', 'timestamp', 'movie_cnt', 'Percentage',
        'label', 'genres_cnt', 'movieGenre1', 'movieGenre2', 'movieGenre3',
        'releaseYear', 'movieRatingCount', 'movieAvgRating', 'movieStdRating',
        'userRatingCnt', 'userAvgRating', 'userRatingStddev',
        'userReleaseYearStddev', 'userAvgReleaseYear', 'userRatedMovie1',
        'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4',
        'userRatedMovie5', 'userGenre1', 'userGenre2', 'userGenre3',
        'userGenre4', 'userGenre5'],
       dtype='object'),
 30,
 Index(['movieId', 'userId', 'rating', 'timestamp', 'movie_cnt', 'Percentage',
        'label', 'genres_cnt', 'movieGenre1', 'movieGenre2', 'movieGenre3',
        'releaseYear', 'movieRatingCount', 'movieAvgRating', 'movieStdRating',
        'userRatingCnt', 'userAvgRating', 'userRatingStddev',
        'userReleaseYearStddev', 'userAvgReleaseYear', 'userRatedMovie1',
        'userRatedMovie2', 'userRatedMovie3', 'userRatedMovi

In [4]:
import tensorflow as tf

# Training samples path, change to your local path
# training_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv",
#                                                      "file:\\D:\\MyProjects\\SparrowRecSys\\src\\main\\resources\\webroot\\sampledata"
#                                                      "\\train.csv")
#                                                     #  "\\trainingSamples.csv")
# # Test samples path, change to your local path
# test_samples_file_path = tf.keras.utils.get_file("testSamples.csv",
#                                                  "file:\\D:\\MyProjects\\SparrowRecSys\\src\\main\\resources\\webroot\\sampledata"
#                                                  "\\test.csv")
#                                                 #  "\\testSamples.csv")


# training_samples_file_path = "../../data/processed_data/train.csv/part-00000-fd08a887-cd12-4275-a8c2-35dd7f171607-c000.csv"
# test_samples_file_path = "../../data/processed_data/test.csv/part-00000-185a5f32-3861-4886-a6ff-3fb5638adc59-c000.csv"

training_samples_file_path = "../../data/trainingSamples.csv"
test_samples_file_path = "../../data/testSamples.csv"



# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset


# split as test dataset and training dataset
train_dataset = get_dataset(training_samples_file_path)
test_dataset = get_dataset(test_samples_file_path)

# define input for keras model
inputs = {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'rating': tf.keras.layers.Input(name='rating', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),

    
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    
    
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),
    'userAvgReleaseYear': tf.keras.layers.Input(name='userAvgReleaseYear', shape=(), dtype='int32'),
    'userReleaseYearStddev': tf.keras.layers.Input(name='userReleaseYearStddev', shape=(), dtype='int32'),
    'timestamp': tf.keras.layers.Input(name='timestamp', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'label': tf.keras.layers.Input(name='label', shape=(), dtype='int32'),
    
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),
    'userRatedMovie2': tf.keras.layers.Input(name='userRatedMovie2', shape=(), dtype='int32'),
    'userRatedMovie3': tf.keras.layers.Input(name='userRatedMovie3', shape=(), dtype='int32'),
    'userRatedMovie4': tf.keras.layers.Input(name='userRatedMovie4', shape=(), dtype='int32'),
    'userRatedMovie5': tf.keras.layers.Input(name='userRatedMovie5', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
}

# movie id embedding feature
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
movie_ind_col = tf.feature_column.indicator_column(movie_col) # movid id indicator columns

# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
user_ind_col = tf.feature_column.indicator_column(user_col) # user id indicator columns

# genre features vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']
# user genre embedding feature
user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="userGenre1",
                                                                           vocabulary_list=genre_vocab)
user_genre_emb_col = tf.feature_column.embedding_column(user_genre_col, 10)
user_genre_ind_col = tf.feature_column.indicator_column(user_genre_col) # user genre indicator columns

# item genre embedding feature
item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="movieGenre1",
                                                                           vocabulary_list=genre_vocab)
item_genre_emb_col = tf.feature_column.embedding_column(item_genre_col, 10)
item_genre_ind_col = tf.feature_column.indicator_column(item_genre_col) # item genre indicator columns




# fm first-order term columns: without embedding and concatenate to the output layer directly
fm_first_order_columns = [movie_ind_col, user_ind_col, user_genre_ind_col, item_genre_ind_col]

deep_feature_columns = [tf.feature_column.numeric_column('releaseYear'),
                        tf.feature_column.numeric_column('movieRatingCount'),
                        tf.feature_column.numeric_column('movieAvgRating'),
                        tf.feature_column.numeric_column('movieRatingStddev'),
                        tf.feature_column.numeric_column('userRatingCount'),
                        tf.feature_column.numeric_column('userAvgRating'),
                        tf.feature_column.numeric_column('userRatingStddev'),
                        movie_emb_col,
                        user_emb_col]

item_emb_layer = tf.keras.layers.DenseFeatures([movie_emb_col])(inputs)
user_emb_layer = tf.keras.layers.DenseFeatures([user_emb_col])(inputs)
# item_genre_emb_layer = tf.keras.layers.DenseFeatures([item_genre_emb_col])(inputs)
# user_genre_emb_layer = tf.keras.layers.DenseFeatures([user_genre_emb_col])(inputs)

item_genre_emb_layer = tf.keras.layers.DenseFeatures([item_genre_emb_col])(inputs)
user_genre_emb_layer = tf.keras.layers.DenseFeatures([user_genre_emb_col])(inputs)


# The first-order term in the FM layer
fm_first_order_layer = tf.keras.layers.DenseFeatures(fm_first_order_columns)(inputs)

# FM part, cross different categorical feature embeddings
product_layer_item_user = tf.keras.layers.Dot(axes=1)([item_emb_layer, user_emb_layer])
product_layer_item_genre_user_genre = tf.keras.layers.Dot(axes=1)([item_genre_emb_layer, user_genre_emb_layer])
product_layer_item_genre_user = tf.keras.layers.Dot(axes=1)([item_genre_emb_layer, user_emb_layer])
product_layer_user_genre_item = tf.keras.layers.Dot(axes=1)([item_emb_layer, user_genre_emb_layer])

# deep part, MLP to generalize all input features
deep = tf.keras.layers.DenseFeatures(deep_feature_columns)(inputs)
deep = tf.keras.layers.Dense(64, activation='relu')(deep)
deep = tf.keras.layers.Dense(64, activation='relu')(deep)

# concatenate fm part and deep part
concat_layer = tf.keras.layers.concatenate([fm_first_order_layer, product_layer_item_user, product_layer_item_genre_user_genre,
                                            product_layer_item_genre_user, product_layer_user_genre_item, deep], axis=1)

# concat_layer = deep

output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(concat_layer)

model = tf.keras.Model(inputs, output_layer)
# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

# train the model
model.fit(next(iter(train_dataset)), epochs=5)

# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))

Epoch 1/5


ValueError: in user code:

    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/wenkanw/.local/lib/python3.8/site-packages/tensorflow/python/keras/e

## Wide&Deep Model

In [5]:
import tensorflow as tf

training_samples_file_path = "../../data/trainingSamples.csv"
test_samples_file_path = "../../data/testSamples.csv"


# training_samples_file_path = "../../data/processed_data/train.csv/part-00000-fd08a887-cd12-4275-a8c2-35dd7f171607-c000.csv"
# test_samples_file_path = "../../data/processed_data/test.csv/part-00000-185a5f32-3861-4886-a6ff-3fb5638adc59-c000.csv"


# training_samples_file_path = tf.keras.utils.get_file("trainingSamples.csv",
#                                                      "file:\\D:\\MyProjects\\SparrowRecSys\\src\\main\\resources\\webroot\\sampledata"
#                                                      "\\train.csv")
#                                                     #  "\\trainingSamples.csv")
# # Test samples path, change to your local path
# test_samples_file_path = tf.keras.utils.get_file("testSamples.csv",
#                                                  "file:\\D:\\MyProjects\\SparrowRecSys\\src\\main\\resources\\webroot\\sampledata"
#                                                  "\\test.csv")

# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=12,
        label_name='label',
        na_value="0",
        num_epochs=1,
        ignore_errors=True)
    return dataset


# split as test dataset and training dataset
train_dataset = get_dataset(training_samples_file_path)
test_dataset = get_dataset(test_samples_file_path)

# genre features vocabulary
genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']

GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab,
    
}

# all categorical features
categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10)
    categorical_columns.append(emb_col)
    
# movie id embedding feature
movie_col = tf.feature_column.categorical_column_with_identity(key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)

# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

# all numerical features
numerical_columns = [tf.feature_column.numeric_column('releaseYear'),
                     tf.feature_column.numeric_column('movieRatingCount'),
                     tf.feature_column.numeric_column('movieAvgRating'),
                     tf.feature_column.numeric_column('movieRatingStddev'),
                     tf.feature_column.numeric_column('userRatingCount'),
                     tf.feature_column.numeric_column('userAvgRating'),
                     tf.feature_column.numeric_column('userRatingStddev')]

# cross feature between current movie and user historical movie
rated_movie = tf.feature_column.categorical_column_with_identity(key='userRatedMovie1', num_buckets=1001)
crossed_feature = tf.feature_column.indicator_column(tf.feature_column.crossed_column([movie_col, rated_movie], 10000))

# define input for keras model
inputs =  {
    'movieAvgRating': tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev': tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'rating': tf.keras.layers.Input(name='rating', shape=(), dtype='float32'),
    'movieRatingCount': tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    
    'userAvgRating': tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev': tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32'),
    'userRatingCount': tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    
    'releaseYear': tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),
    'userAvgReleaseYear': tf.keras.layers.Input(name='userAvgReleaseYear', shape=(), dtype='int32'),
    'userReleaseYearStddev': tf.keras.layers.Input(name='userReleaseYearStddev', shape=(), dtype='int32'),
    'timestamp': tf.keras.layers.Input(name='timestamp', shape=(), dtype='int32'),

    'movieId': tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId': tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    
    'userRatedMovie1': tf.keras.layers.Input(name='userRatedMovie1', shape=(), dtype='int32'),
    'userRatedMovie2': tf.keras.layers.Input(name='userRatedMovie2', shape=(), dtype='int32'),
    'userRatedMovie3': tf.keras.layers.Input(name='userRatedMovie3', shape=(), dtype='int32'),
    'userRatedMovie4': tf.keras.layers.Input(name='userRatedMovie4', shape=(), dtype='int32'),
    'userRatedMovie5': tf.keras.layers.Input(name='userRatedMovie5', shape=(), dtype='int32'),

    'userGenre1': tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2': tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3': tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4': tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5': tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    
    'movieGenre1': tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2': tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3': tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
}


# wide and deep model architecture
# deep part for all input features
deep = tf.keras.layers.DenseFeatures(numerical_columns + categorical_columns)(inputs)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)

# wide part for cross feature
wide = tf.keras.layers.DenseFeatures(crossed_feature)(inputs)
both = tf.keras.layers.concatenate([deep, wide])
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(both)
model = tf.keras.Model(inputs, output_layer)

# compile the model, set loss function, optimizer and evaluation metrics
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])

# train the model
model.fit(train_dataset, epochs=5)

# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))


Epoch 1/5
7403/7403 [==============================] - 83s 11ms/step - loss: 0.7497 - accuracy: 0.6146 - auc_4: 0.6362 - auc_5: 0.6694
Epoch 2/5
7403/7403 [==============================] - 80s 11ms/step - loss: 0.6025 - accuracy: 0.6796 - auc_4: 0.7318 - auc_5: 0.7558
Epoch 3/5
7403/7403 [==============================] - 80s 11ms/step - loss: 0.5481 - accuracy: 0.7215 - auc_4: 0.7904 - auc_5: 0.8113
Epoch 4/5
7403/7403 [==============================] - 80s 11ms/step - loss: 0.5081 - accuracy: 0.7522 - auc_4: 0.8243 - auc_5: 0.8453
Epoch 5/5
1870/1870 [==============================] - 15s 7ms/step - loss: 0.6121 - accuracy: 0.6924 - auc_4: 0.7537 - auc_5: 0.7835


Test Loss 0.6121305823326111, Test Accuracy 0.6923796534538269, Test ROC AUC 0.7536923885345459, Test PR AUC 0.7834892868995667
Predicted good rating: 79.54%  | Actual rating label:  Bad Rating
Predicted good rating: 2.82%  | Actual rating label:  Good Rating
Predicted good rating: 58.88%  | Actual rating label:  Good Rati

In [51]:
# import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


# DeepFM model
+ Note
    - After adding FM part, the model overfits the training set (better than Wide&Deep model) and lower the AUC , Accuracy score in test set

In [158]:
import tensorflow as tf

training_samples_file_path = "../../data/trainingSamples.csv"
test_samples_file_path = "../../data/testSamples.csv"


# training_samples_file_path = "../../data/processed_data/train.csv/part-00000-01c21c89-bb0b-47ee-a791-64d019d8cb60-c000.csv"
# test_samples_file_path = "../../data/processed_data/test.csv/part-00000-2f04f2e9-c2c8-4918-b79d-88fdee316947-c000.csv"



def get_dataset(file_path):
    """
    Input: 
        - file_path : path to csv file 
    Output:
        - tensorflow Prefetch dataset, which is to read by iterator
          with next(iter(dataset))[0] = a batch of sample in dictionary format,
          next(iter(dataset))[1] = corresponding label vector
          
    """
    dataset = tf.data.experimental.make_csv_dataset(file_path,
                                       batch_size = 32,
                                       na_value = '0',
                                       ignore_errors= True,
                                       label_name = 'label',
                                       num_epochs = 1)
    return dataset


train_set = get_dataset(training_samples_file_path)
test_dataset = get_dataset(test_samples_file_path)
col_string = []
col_int32 = []
col_float32 = []
inputs = {}

genre_vocab = ['Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary',
               'Sci-Fi', 'Drama', 'Thriller',
               'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical']


# construct feature columns to transform features in tensorflow model
movie_col = tf.feature_column.categorical_column_with_identity(key= 'movieId', num_buckets = 1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
movie_ind_col = tf.feature_column.indicator_column(movie_col)

user_col = tf.feature_column.categorical_column_with_identity(key='userId', num_buckets = 30001)
user_emb_col = tf.feature_column.embedding_column(user_col,10)

rated_movie_cols = []
rated_movie_emb_cols = []
for i in range(1, 6):
    rated_movie_col = tf.feature_column.categorical_column_with_identity(key='userRatedMovie'+ str(i), num_buckets = 1001)
    rated_movie_cols.append(rated_movie_col)
    rated_movie_emb_cols.append(tf.feature_column.embedding_column(rated_movie_col,10))  

item_genre_cols = []
item_genre_emb_cols = []
for i in range(1, 4):
    item_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="movieGenre"+str(i),
                                                                               vocabulary_list=genre_vocab)
    item_genre_cols.append(item_genre_col)
    item_genre_emb_cols.append( tf.feature_column.embedding_column(item_genre_col, 10))
    
    
user_genre_cols = []
user_genre_emb_cols = []
for i in range(1, 6):
    user_genre_col = tf.feature_column.categorical_column_with_vocabulary_list(key="userGenre"+str(i),
                                                                               vocabulary_list=genre_vocab)
    user_genre_cols.append(user_genre_col)
    user_genre_emb_cols.append( tf.feature_column.embedding_column(user_genre_col, 10))

print(user_genre_emb_cols[0])
user_genre_emb_layers = user_genre_emb_cols

item_genre_ind_col = tf.feature_column.indicator_column(item_genre_col) # item genre indicator columns
user_genre_ind_col = tf.feature_column.indicator_column(user_genre_col) # item genre indicator columns


deep_columns =  [ #tf.feature_column.numeric_column("movie_cnt"),
                  tf.feature_column.numeric_column("movieAvgRating"),
                  tf.feature_column.numeric_column("movieRatingCount"),
                  tf.feature_column.numeric_column("movieRatingStddev"),
                  #tf.feature_column.numeric_column("Percentage"), 
                  
                  tf.feature_column.numeric_column("userRatingCount"),
                  tf.feature_column.numeric_column("userAvgRating"),
                  tf.feature_column.numeric_column("userRatingStddev"),
                  
                  tf.feature_column.numeric_column("releaseYear"),
                  movie_emb_col,
                  user_emb_col,
                  *item_genre_emb_cols,
                  *rated_movie_emb_cols
              ]

fm_first_order_columns = [movie_ind_col, rated_movie_emb_cols[0],
                         user_genre_ind_col,
#                   tf.feature_column.numeric_column("movieAvgRating"),
#                   tf.feature_column.numeric_column("movieRatingCount"),
#                   tf.feature_column.numeric_column("movieRatingStddev"),
#                   tf.feature_column.numeric_column("userRatingCount"),
#                   tf.feature_column.numeric_column("userAvgRating"),
#                   tf.feature_column.numeric_column("userRatingStddev"),
                  ]





# deep_columns =  [ #tf.feature_column.numeric_column("movie_cnt"),
#                   tf.feature_column.numeric_column("movieAvgRating"),
#                   tf.feature_column.numeric_column("movieRatingCount"),
#                   tf.feature_column.numeric_column("movieStdRating"),
#                   #tf.feature_column.numeric_column("Percentage"), 
                  
#                   tf.feature_column.numeric_column("userRatingCnt"),
#                   tf.feature_column.numeric_column("userAvgRating"),
#                   tf.feature_column.numeric_column("userRatingStddev"),
                  
#                   tf.feature_column.numeric_column("releaseYear"),
#                   movie_emb_col,
#                   user_emb_col,
#                   #*item_genre_emb_cols,
#                   #*rated_movie_emb_cols
#               ]

# fm_first_order_columns = [#movie_emb_col,user_emb_col,
#                   tf.feature_column.numeric_column("movieAvgRating"),
#                   tf.feature_column.numeric_column("movieRatingCount"),
#                   tf.feature_column.numeric_column("movieStdRating"),
#                   tf.feature_column.numeric_column("userRatingCnt"),
#                   tf.feature_column.numeric_column("userAvgRating"),
#                   tf.feature_column.numeric_column("userRatingStddev"),
#                   ]


# build  Input layer
for k, v in next(iter(train_set))[0].items():
    if v.dtype == "string" or  v.dtype == "object":
        col_string.append(k)
    elif  v.dtype == "int32":
        col_int32.append(k)
    elif v.dtype == "float32":
        col_float32.append(k)
    print(k, v.dtype)
#     if k == 'userAvgReleaseYear':
#         print(v)
    if k != "label":
        #print("Key", k)
        inputs[k]= tf.keras.layers.Input(name= k, shape=(), dtype=v.dtype)
        
# Build model layer

rs = tf.keras.layers.Reshape((10,1))

item_emb_layer = rs(tf.keras.layers.DenseFeatures([movie_emb_col])(inputs))
user_emb_layer = rs(tf.keras.layers.DenseFeatures([user_emb_col])(inputs))
# rated_movie_emb_layer = rs(tf.keras.layers.DenseFeatures([rated_movie_emb_col])(inputs))
# item_genre_emb_layer = rs(tf.keras.layers.DenseFeatures([item_genre_emb_col])(inputs))

for i in range(len(item_genre_emb_cols)):
    item_genre_emb_cols[i] = rs(tf.keras.layers.DenseFeatures([item_genre_emb_cols[i]])(inputs))
    
for i in range(len(rated_movie_emb_cols)):
    rated_movie_emb_cols[i] = rs(tf.keras.layers.DenseFeatures([rated_movie_emb_cols[i]])(inputs))

for i in range(len(user_genre_emb_cols)):
    user_genre_emb_cols[i] = rs(tf.keras.layers.DenseFeatures([user_genre_emb_cols[i]])(inputs))


# user_genre_emb_layer = tf.keras.layers.DenseFeatures([user_genre_emb_col])(inputs)

# FM deep part
deep = tf.keras.layers.DenseFeatures(deep_columns)(inputs)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)
deep = tf.keras.layers.Dense(128, activation='relu')(deep)

# FM layer
# FM first order
fm_first_order_layer = tf.keras.layers.DenseFeatures(fm_first_order_columns)(inputs)


user_genre_emb_mean = sum(user_genre_emb_cols)/len(user_genre_emb_cols)
rated_movie_emb_mean = sum(rated_movie_emb_cols)/len(rated_movie_emb_cols)
item_genre_emb_mean = sum(item_genre_emb_cols)/len(item_genre_emb_cols)
# FM second order
fm_feature_mat = tf.keras.layers.Concatenate(axis=2)([item_emb_layer, 
                                                      user_emb_layer,
                                                      user_genre_emb_mean,
                                                      rated_movie_emb_mean,           
                                                      item_genre_emb_mean,
                                                     ])
square_of_sum = tf.square(tf.reduce_sum(fm_feature_mat, axis=2, keepdims=True))
sum_of_square = tf.reduce_sum(fm_feature_mat * fm_feature_mat, axis=2, keepdims=True)
cross_term = square_of_sum - sum_of_square
cross_term = 0.5 * tf.reduce_sum(cross_term, axis=2, keepdims=False)

## Original cross term by SparrowRec source code
# product_layer_item_user = tf.keras.layers.Dot(axes=1)([item_emb_layer, user_emb_layer])
# product_layer_item_genre_user_genre = tf.keras.layers.Dot(axes=1)([item_genre_emb_layer, user_genre_emb_layer])
# product_layer_item_genre_user = tf.keras.layers.Dot(axes=1)([item_genre_emb_layer, user_emb_layer])
# product_layer_user_genre_item = tf.keras.layers.Dot(axes=1)([item_emb_layer, user_genre_emb_layer])



concat_layer = tf.keras.layers.concatenate([fm_first_order_layer,cross_term, deep])
out = tf.keras.layers.Dense(1, activation="sigmoid")(concat_layer)



metrics=['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')]

model = tf.keras.Model(inputs, out)
with tf.device('/GPU:0'):
    model.compile(loss="binary_crossentropy",
                 optimizer= "adam",
                 metrics = ['accuracy', tf.keras.metrics.AUC(curve='ROC'), tf.keras.metrics.AUC(curve='PR')])
    history = model.fit(train_set, epochs=5)
    print(model.predict(next(iter(train_set))[0]).shape)


# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(test_dataset)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.format(test_loss, test_accuracy,
                                                                                   test_roc_auc, test_pr_auc))

# print some predict results
predictions = model.predict(test_dataset)
for prediction, goodRating in zip(predictions[:12], list(test_dataset)[0][1][:12]):
    print("Predicted good rating: {:.2%}".format(prediction[0]),
          " | Actual rating label: ",
          ("Good Rating" if bool(goodRating) else "Bad Rating"))
    
    
# ls = ['movieId', 'userId', 'rating', 'timestamp', 'movie_cnt', 'Percentage',
#         'label', 'genres_cnt', 'releaseYear', 'movieRatingCount',
#         'movieAvgRating', 'movieStdRating', 'userRatingCnt', 'userAvgRating',
#         'userRatingStddev', 'userReleaseYearStddev', 'userAvgReleaseYear',
#         'userRatedMovie1', 'userRatedMovie2', 'userRatedMovie3',
#         'userRatedMovie4', 'userRatedMovie5', 'userGenre1', 'userGenre2',
#         'userGenre3', 'userGenre4', 'userGenre5']
        
    
## Note: 
# after adding user_emb in FM, model is overfitting easily


EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='userGenre1', vocabulary_list=('Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy', 'Western', 'Documentary', 'Sci-Fi', 'Drama', 'Thriller', 'Crime', 'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical'), dtype=tf.string, default_value=-1, num_oov_buckets=0), dimension=10, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x1527e0b2da00>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True, use_safe_embedding_lookup=True)
movieId <dtype: 'int32'>
userId <dtype: 'int32'>
rating <dtype: 'float32'>
timestamp <dtype: 'int32'>
releaseYear <dtype: 'int32'>
movieGenre1 <dtype: 'string'>
movieGenre2 <dtype: 'string'>
movieGenre3 <dtype: 'string'>
movieRatingCount <dtype: 'int32'>
movieAvgRating <dtype: 'float32'>
movieRatingStddev <dtype: 'float32'>
userRatedMovie1 <dtype: 'int32'>
userRatedMovie2 <dtype: 'int32'>
userRatedMovi

tf.Tensor([0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 

tf.Tensor([1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 0 1 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 

tf.Tensor([1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 1 0 0 0 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 1 1 0 0 0 0 0 1 1 1 0 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 1 0 1 1 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 1 0 

tf.Tensor([0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 0 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 0 1 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 0 

tf.Tensor([0 0 0 1 1 1 0 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 0 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 1 0 0 1 0 1 1 0 0 1 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 1 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 1 1 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 

tf.Tensor([0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 0 1 0 0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 0 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 0 1 0 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 1 0 

tf.Tensor([1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 0 1 0 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 0 0 1 1 1 0 1 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 0 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 1 1 1 0 0 1 1 0 

tf.Tensor([0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 1 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 0 1 1 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 0 1 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 0 

tf.Tensor([0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 1 1 0 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 0 1 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 0 1 0], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1 1 

tf.Tensor([0 0 0 1 0 0 0 1 1 1 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 1 1 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 1 0 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 1 1], shape=(32,), dtype=int32)
tf.Tensor([0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([0 1 1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0], shape=(32,), dtype=int32)
tf.Tensor([0 0 1 0 0 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 1 1 0 1 1 0 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 0 1], shape=(32,), dtype=int32)
tf.Tensor([1 0 1 1 0 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 